In [1]:
# Importando Librerías
import re
import json
import time
import pickle
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)


In [2]:
options = Options()
# options.add_argument("start-maximized")

In [3]:
# Links
# 2020: https://www.ceroacero.es/edition.php?id_edicao=141620
    # Jornada 1: https://www.ceroacero.es/edition.php?jornada_in=1&id_edicao=141620&fase=136763

In [4]:
driver = webdriver.Chrome(
        executable_path='web_driver/chromedriver.exe', options=options)

In [5]:
driver.get('https://www.ceroacero.es/edition.php?jornada_in=9&id_edicao=153891&fase=146778')

In [6]:
cookies = pickle.load(open("cookies/ceroacero_cookies_login.pkl", "rb"))

for cookie in cookies:
    driver.add_cookie(cookie)

In [7]:
driver.get('https://www.ceroacero.es/edition.php?jornada_in=9&id_edicao=153891&fase=146778')

In [ ]:
t = driver.find_element(By.CLASS_NAME, "box_container")
list(map(str.strip, t.text.split("\n")))

In [ ]:
t = driver.find_element(By.CLASS_NAME, "box_container")
results = list(map(str.strip, t.text.split("\n")))
results

In [ ]:
# Obtener Datos de los Partidos: Fecha - Equipo Local - Equipo Visitante - Goles Locales - Goles Visitante
games_date = []
for game in results:
    if bool(re.match(r"[\d][\d]", game)):
        date_game = game[:5]
        game = game[6:]

        # Nombres de los equipos
        teams_score = game.split('-')
        if len(teams_score)<2:
            teams_score = game.split(' DA ')

        # print(teams_score)
        name_local = ' '.join(teams_score[0].split()[:-1])
        name_away = ' '.join(teams_score[1].split()[1:-1])

        # Resultados
        score_local = re.findall(r'\d+', teams_score[0])[0]
        score_away = re.findall(r'\d+', teams_score[1])[0]

        games_date.append([date_game, name_local,
                            name_away, int(score_local), int(score_away)])
    else:
        # Nombres de los equipos
        teams_score = game.split('-')
        if len(teams_score)<2:
            teams_score = game.replace(' DA ', ' 3-0 ').split('-')
            
        # print(teams_score)
        name_local = ' '.join(teams_score[0].split()[:-1])
        name_away = ' '.join(teams_score[1].split()[1:-1])

        # Resultados
        score_local = re.findall(r'\d+', teams_score[0])[0]
        score_away = re.findall(r'\d+', teams_score[1])[0]

        games_date.append([date_game, name_local,
                            name_away, int(score_local), int(score_away)])
    
    print([date_game, name_local,name_away, int(score_local), int(score_away)])

In [ ]:
# Ingresando a página para obtener información de los partidos de futbol la Liga 1 Peruana 2020
page_url = f'https://www.ceroacero.es/edition.php?jornada_in=1&id_edicao=141620&fase=136763'
driver.get(page_url)

cookies = pickle.load(open("cookies/ceroacero_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.refresh()

In [ ]:
# Resultados de los Partidos
results = driver.find_element(By.CLASS_NAME, "box_container")
results = list(map(str.strip, results.text.split("\n")))
print(results)


In [ ]:
for index, value in enumerate(results):
    print(index, value)

In [ ]:
# Obtener fecha del Partido
games_date = []
for game in results:
    if bool(re.match(r"[\d][\d]", game)):
        date_game = game[:5]
        game = game[6:]

        # Nombres de los equipos
        teams_score = game.split('-')

        name_local = ' '.join(teams_score[0].split()[:-1])
        name_away = ' '.join(teams_score[1].split()[1:-1])

        # Resultados
        score_local = re.findall(r'\d+', teams_score[0])[0]
        score_away = re.findall(r'\d+', teams_score[1])[0]

        games_date.append([date_game, name_local,
                          name_away, int(score_local), int(score_away)])
    else:
        # Nombres de los equipos
        teams_score = game.split('-')

        name_local = ' '.join(teams_score[0].split()[:-1])
        name_away = ' '.join(teams_score[1].split()[1:-1])

        # Resultados
        score_local = re.findall(r'\d+', teams_score[0])[0]
        score_away = re.findall(r'\d+', teams_score[1])[0]

        games_date.append([date_game, name_local,
                          name_away, int(score_local), int(score_away)])

games_date


In [ ]:
game_links = driver.find_elements(By.CLASS_NAME, "result > a")
game_links = game_links[:len(game_links)//2]

for game_link in game_links:
    print(game_link.get_attribute("href"))

In [ ]:
local = driver.find_elements(By.CLASS_NAME, 'home')
local[0].text.split('\n')

In [ ]:
stats = driver.find_element(By.XPATH, '//*[@id="page_main"]/div[1]/table').text.split('\n')[1]
# stats.find(' ')
stats.split()

In [ ]:
temp = game_links[1].get_attribute("href")
driver.get(temp)

In [ ]:
headers = driver.find_elements(By.XPATH, '//*[@id="page_main"]/div[1]/table/tbody/tr[2]/td/span')
for i in headers:
    print(i.text)

In [ ]:
stats = pd.read_html(temp.replace('match','match_performance'))
stats

In [ ]:
stats[2].rename(columns={'Unnamed: 0':'Jugadores'})

In [ ]:
stats[3].rename(columns={'Unnamed: 0':'Jugadores'})

In [ ]:
stats = driver.find_element(By.XPATH, '//*[@id="page_rightbar"]/div[3]/div/div[2]')
all_stats = stats.text.split('\n')
all_stats = [stats.strip(' ') for stats in all_stats]
all_stats = [all_stats[i:i+3] for i in range(0, len(all_stats), 3)]
all_stats

In [5]:
driver.get('https://www.ceroacero.es/match.php?id=7324824')

In [22]:
stats = driver.find_element(
    By.XPATH, '//*[@id="page_rightbar"]/div[3]/div/div[2]')
all_stats = stats.text.split('\n')
all_stats = [stats.strip(' ') for stats in all_stats]
all_stats = {all_stats[i]:all_stats[i+1:i+3] for i in range(0, len(all_stats), 3)}

In [23]:
all_stats

{'REMATES A PUERTA': ['2', '1'],
 'TIROS': ['6', '8'],
 'FALTAS': ['19', '19'],
 'SAQUES DE ESQUINA': ['1', '7'],
 'POSESIÓN': ['53 %', '47 %']}

In [6]:
performance = 'https://www.ceroacero.es/match.php?id=8066906'

In [7]:
stats = pd.read_html(performance.replace('match','match_performance'))
stats

[                      0                          1                   2  \
 0                LaLiga  Robert Lewandowski8 Goles  Real Madrid+2 Pts.   
 1          Liga Inglesa     Erling Haaland11 Goles      Arsenal+1 Pts.   
 2         Liga Francesa              Neymar8 Goles     Paris SG+2 Pts.   
 3  Campeonato Brasileño        Germán Cano15 Goles    Palmeiras+8 Pts.   
 
                          3  
 0   5 DerrotasMás Derrotas  
 1   6 DerrotasMás Derrotas  
 2   6 DerrotasMás Derrotas  
 3  15 DerrotasMás Derrotas  ,
                 Unnamed: 0 Equipo P:   M  V  PP  AST:  A  R  PENALTI  PP:  PF:
 0          Ignacio Barrios   CAAS  P  90  0   0     0  0  0        0    0    0
 1              Adán Balbín   CAAS  D  90  0   0     0  0  0        0    0    0
 2            Carlos Ascues   CAAS  D  78  0   0     0  0  0        0    0    0
 3             Juan Barreda   CAAS  D  90  0   0     0  0  0        0    0    0
 4          Jesús Arismendi   CAAS  D  90  0   0     0  0  0        0   

In [ ]:
dataTables_scroll

In [5]:
driver.get('https://www.ceroacero.es/match_performance.php?id=8066907')